In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv))

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
import pathlib
from fastai.basics import L
from ast import literal_eval
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
from fastai.vision.all import *

In [ ]:
train_imgs_path = pathlib.Path('/kaggle/input/global-wheat-detection/train')
test_imgs_path = pathlib.Path('/kaggle/input/global-wheat-detection/test/')
train_img_bbox_df = pd.read_csv(pathlib.Path('/kaggle/input/global-wheat-detection/train.csv'), delimiter=',')
train_img_bbox_df.shape

In [ ]:
train_img_bbox_df


In [ ]:
train_img_ids = L([im[:-4] for im in os.listdir(train_imgs_path) if im.find('.jpg') != -1])
train_img_ids

In [ ]:
test_img_ids = L([im[:-4] for im in os.listdir(test_imgs_path) if im.find('.jpg') != -1])
test_img_ids

In [ ]:
def image2bbox(img_id, image_dir_path):
    bbox = train_img_bbox_df[train_img_bbox_df['image_id']==img_id]['bbox'].apply(lambda x: literal_eval(x))
    img_name=img_id+'.jpg'
    img_path = pathlib.Path(image_dir_path/img_name)
    labels =[]
    if len(bbox) > 0:
        bbox = bbox.apply(lambda x: [a if idx <2 else a+x[idx-2] for idx,a in enumerate(x)]) #dataset has boboxes encoded [x_topleft, y_topleft, width, height]
        labels = ['True']*len(bbox) 

    return img_path, bbox.tolist(), labels

# test for an image
i, b,l = image2bbox('b6ab77fd7', train_imgs_path)
raw_bbox = train_img_bbox_df[train_img_bbox_df['image_id']=='b6ab77fd7']['bbox']

In [ ]:
print(b[0])
print(raw_bbox[0])

In [ ]:
data_dict ={k:image2bbox(k, train_imgs_path) for k in train_img_ids}

In [ ]:
print('Data dict size (num unique images): ', len(data_dict))
positives_dict ={ k : v for k,v in data_dict.items() if len(v[1])>0 }
negatives_dict= { k: v for k,v in data_dict.items() if len(v[1])==0 }
print ('Num negatives: ', len(negatives_dict))
print('Num Positives: ', len(positives_dict))


In [ ]:
train_db = DataBlock(blocks = (ImageBlock, BBoxBlock, BBoxLblBlock),
                    get_items=get_image_files,
                    splitter= RandomSplitter(),
                    get_y= [lambda o:data_dict[o.name[:-4]][1], lambda o:data_dict[o.name[:-4]][2]],
                    item_tfms =  [Resize(512, method='resize')],
                    batch_tfms = [Normalize()],
                    n_inp=1)
train_db.summary(train_imgs_path)

In [ ]:
dls=train_db.dataloaders(train_imgs_path,bs=1, verbose=True)
train_dl = dls.train

In [ ]:
from PIL import Image
def show_sample_batch(dl, max_n=2):
    x,y1,y2=next(iter(dl))
    print(x.shape)
    print(y1.shape)
    print(y2.shape)
    xd,y1d,y2d = dl.decode((x,y1,y2))
    xd = np.uint8(xd.numpy().squeeze())
    xd = np.transpose(xd, axes=[1,2,0])
    print(xd.shape)
    return xd, y1d, y2d

import matplotlib.pyplot as plt
%matplotlib inline

img, y1, y2 = show_sample_batch(train_dl)
ax = plt.imshow(img)



In [ ]:
#Lets try to put the RetinaNet together using stuff from all around
backbone = create_body(resnet18, pretrained=True, cut=-2)
backbone